# **Data Cleaning - Ames Housing dataset**

## Objectives

* Load relevant .csv data file from Kaggle
* Handle missing values appropriately 
* Fix bad data: Empty cells, Data in wrong format, Wrong data, Duplicates
* Detect and treat outliers 
* Remove duplicate rows if any exist 
* Standardise categorical values where necessary 
* Save cleaned data for modelling 

## Inputs

* Datasets:
   - Raw datset: data/raw/house_prices_records.csv 
   - Client dataset: data/raw/inherited_houses.csv 
* Libraries pandas, numpy

## Outputs

* Updated variables and explanations that will be used for the EDA, feature selection and ML model. 
* Cleaned dataset saved to: data/processed/cleaned_data.csv 

## Additional Comments

* Doccument all cleaning with justified decisions
* Visualise missing values before and after cleaning 
* The cleaned dataset will be resused in later notebooks 


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/Users/aisha/Desktop/vscode-projects/p5-heritage-housing/p5-heritage-housing/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/Users/aisha/Desktop/vscode-projects/p5-heritage-housing/p5-heritage-housing'

---

# Data Cleaning Introduction 

In this notebook, I will clean the Ames Housing dataset to prepare it for exploratory data analysis (EDA) and machine learning (ML).

#### 1. Inspect the raw structure of the dataset
- Confirm the dataset shape
- Ensure the dataset loaded correctly before cleaning.

#### 2. Identify and handle any duplicate rows 
- Detect any fully duplicated entries.
- Decide whether these should be removed to prevent bias during training.

#### 3. Analyse missing data patterns
- Identify columns with high levels of missingness.
- Group columns by type of missingness 

#### 4. Decide on appropriate missing value treatments
Based on the nature of each feature:
- Impute numerical columns using mean/median where appropriate.
- Impute categorical columns using mode or a meaningful label 
- Drop columns with extremely high missingness if they provide no predictive value.
- Consider dropping rows with excessive missing values if justified.

#### 5. Validate and correct data types
- Convert incorrectly stored numeric columns (e.g., stored as object) to integers/floats.
- Ensure categorical features are stored as object or category.
- Handle year-based columns to ensure no impossible dates.

#### 6. Detect and treat outliers
- Use summary statistics to identify unrealistic values 
- Decide whether to cap, remove, or leave outliers 

#### 7. Standardise and clean categorical values
- Check for inconsistent spelling or formatting.
- Replace values such as “None”/“NA”/“Missing” with consistent labels.
- Ensure categories match between the main dataset and inherited_houses.csv.

#### 8. Produce and save a clean dataset
- Once all cleaning steps are complete, export the final dataset as cleaned_data.csv into a data/processed/ folder.
- This cleaned dataset will feed into the next notebooks (EDA and Model Training).


---

# Data Cleaning 

### Step 1: Load the raw datasets and inspect their structures

Even though the dataset was previously explored in 01_data_exploration.ipynb, I will briefly inspect it again to confirm the structure before cleaning. 

This helps ensure that:

- The files loaded correctly.
- Column names and dtypes are as expected.
- The cleaning decisions made are based on the current state of the raw data.

In [7]:
import pandas as pd
import numpy as np

# Display all columns when viewing DataFrames
pd.set_option("display.max_columns", None)

# Load datasets
housing_df = pd.read_csv("data/raw/house_prices_records.csv")
inherited_df = pd.read_csv("data/raw/inherited_houses.csv")

# Preview of the main dataset
housing_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,GarageYrBlt,GrLivArea,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,2003.0,1710,Gd,8450,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,1976.0,1262,TA,9600,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,2001.0,1786,Gd,11250,68.0,162.0,42,5,7,920,NaN,2001,2002,223500
3,961,NaN,NaN,No,216,ALQ,540,NaN,642,Unf,1998.0,1717,Gd,9550,60.0,0.0,35,5,7,756,NaN,1915,1970,140000
4,1145,NaN,4.0,Av,655,GLQ,490,0.0,836,RFn,2000.0,2198,Gd,14260,84.0,350.0,84,5,8,1145,NaN,2000,2000,250000


In [19]:
# Preview of the Client dataset
inherited_df.head()

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,GarageYrBlt,GrLivArea,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,896,0,2,No,468.0,Rec,270.0,0,730.0,Unf,1961.0,896,TA,11622,80.0,0.0,0,6,5,882.0,140,1961,1961
1,1329,0,3,No,923.0,ALQ,406.0,0,312.0,Unf,1958.0,1329,Gd,14267,81.0,108.0,36,6,6,1329.0,393,1958,1958
2,928,701,3,No,791.0,GLQ,137.0,0,482.0,Fin,1997.0,1629,TA,13830,74.0,0.0,34,5,5,928.0,212,1997,1998
3,926,678,3,No,602.0,GLQ,324.0,0,470.0,Fin,1998.0,1604,Gd,9978,78.0,20.0,36,6,6,926.0,360,1998,1998


The above confirms that the files loaded correctly and allows me to visually inspect the first few rows of each dataset.

In [14]:
# Shape of main dataset

print("Shape of housing_df:", housing_df.shape)
print("\nInfo for housing_df:")
housing_df.info()

Shape of housing_df: (1460, 24)

Info for housing_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       1460 non-null   int64  
 1   2ndFlrSF       1374 non-null   float64
 2   BedroomAbvGr   1361 non-null   float64
 3   BsmtExposure   1422 non-null   object 
 4   BsmtFinSF1     1460 non-null   int64  
 5   BsmtFinType1   1315 non-null   object 
 6   BsmtUnfSF      1460 non-null   int64  
 7   EnclosedPorch  136 non-null    float64
 8   GarageArea     1460 non-null   int64  
 9   GarageFinish   1225 non-null   object 
 10  GarageYrBlt    1379 non-null   float64
 11  GrLivArea      1460 non-null   int64  
 12  KitchenQual    1460 non-null   object 
 13  LotArea        1460 non-null   int64  
 14  LotFrontage    1201 non-null   float64
 15  MasVnrArea     1452 non-null   float64
 16  OpenPorchSF    1460 non-null   int64  
 17

`housing_df.shape` shows the number of rows and columns in the dataset.

`housing_df.info()` displays:
- Each column name
- Data type (numeric, object, etc.)
- Number of non-null values

This information helps identify:
- Columns with missing values
- Columns with incorrect data types that may need conversion

In [16]:
# Shape of client dataset

print("Shape of inherited_df:", inherited_df.shape)
print("\nInfo for inherited_df:")
inherited_df.info()

inherited_df.head()

Shape of inherited_df: (4, 23)

Info for inherited_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   1stFlrSF       4 non-null      int64  
 1   2ndFlrSF       4 non-null      int64  
 2   BedroomAbvGr   4 non-null      int64  
 3   BsmtExposure   4 non-null      object 
 4   BsmtFinSF1     4 non-null      float64
 5   BsmtFinType1   4 non-null      object 
 6   BsmtUnfSF      4 non-null      float64
 7   EnclosedPorch  4 non-null      int64  
 8   GarageArea     4 non-null      float64
 9   GarageFinish   4 non-null      object 
 10  GarageYrBlt    4 non-null      float64
 11  GrLivArea      4 non-null      int64  
 12  KitchenQual    4 non-null      object 
 13  LotArea        4 non-null      int64  
 14  LotFrontage    4 non-null      float64
 15  MasVnrArea     4 non-null      float64
 16  OpenPorchSF    4 non-null      int64  
 17  Ove

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,GarageYrBlt,GrLivArea,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,896,0,2,No,468.0,Rec,270.0,0,730.0,Unf,1961.0,896,TA,11622,80.0,0.0,0,6,5,882.0,140,1961,1961
1,1329,0,3,No,923.0,ALQ,406.0,0,312.0,Unf,1958.0,1329,Gd,14267,81.0,108.0,36,6,6,1329.0,393,1958,1958
2,928,701,3,No,791.0,GLQ,137.0,0,482.0,Fin,1997.0,1629,TA,13830,74.0,0.0,34,5,5,928.0,212,1997,1998
3,926,678,3,No,602.0,GLQ,324.0,0,470.0,Fin,1998.0,1604,Gd,9978,78.0,20.0,36,6,6,926.0,360,1998,1998


The summary above shows the structure of the client's dataset. 

I check:

- whether all expected columns are present,
- which columns contain missing values,
- and if any data types differ from those in the main dataset.

### Step 2: Check for duplicate rows

Duplicate rows can distort both exploratory analysis (EDA) and machine learning (ML) models. 
If the same property appears more than once in the dataset, it could artificially inflate patterns and reduce model accuracy.

In this step, I will:
- Count how many exact duplicate rows exist in the main `housing_df` dataset.
- Decide whether duplicates need to be removed in the next cleaning step.

**This step is only being applied to the main dataset (housing_df), because duplicate inherited houses don’t matter since the client only has 4 unique houses.**

In [21]:
# Count duplicate rows in the main dataset

duplicate_count = housing_df.duplicated().sum()
duplicate_count

np.int64(0)

The output above shows there are no duplicates in the dataset and no action is needed.

### Step 3: Missing Values Analysis 

I previously explored missing values in 01_data_exploration.ipynb, where I identified which columns contained missing data. In this section, I will briefly re-check the missing values. This check provides:
- a quick verification that the dataset still contains the same missing values,
- a summary table to guide the cleaning process,
- confirmation that the `inherited_houses.csv` dataset contains no missing values.

In [23]:
# Missing values summary for the main dataset

missing_counts = housing_df.isnull().sum()
missing_percent = (missing_counts / len(housing_df)) * 100

missing_table = (
    pd.DataFrame({
        "missing_count": missing_counts,
        "missing_percent": missing_percent
    })
    .query("missing_count > 0")
    .sort_values(by="missing_percent", ascending=False)
)

missing_table

,missing_count,missing_percent
EnclosedPorch,1324,90.684932
WoodDeckSF,1305,89.383562
LotFrontage,259,17.739726
GarageFinish,235,16.095890
BsmtFinType1,145,9.931507
BedroomAbvGr,99,6.780822
2ndFlrSF,86,5.890411
GarageYrBlt,81,5.547945
BsmtExposure,38,2.602740
MasVnrArea,8,0.547945


The table above summarises all columns in the main dataset that contain missing values.

Since I already investigated the meaning of these missing patterns in 01_data_exploration.ipynb, the purpose here is simply to confirm:
- how many values need to be handled,
- which columns require imputation,
- which columns may represent "not applicable" rather than true missing data.

In [24]:
inherited_df.isna().sum().sum()

np.int64(0)

The inherited_houses.csv dataset contains no missing values, as confirmed above. No cleaning is required for this dataset at this stage.

### Step 4: Handling Missing Values

Using the summary table before, I will now clean the missing values in the main housing_df dataset.  

The strategy is different depending on the type and importance of each feature:

- Drop features with high percentages of missing values.
- Impute numeric features using the median to avoid being affected by outliers.
- Impute categorical features where missing values represent the absence of a feature (e.g. no garage, no basement).
- Re-check the dataset afterwards to confirm that all missing values have been handled.

These decisions are made to keep as much useful information as possible, while avoiding noise and bias from poorly populated features.

In [26]:
# 4.2 Drop features with high missingness

cols_to_drop = ["EnclosedPorch", "WoodDeckSF"]

housing_df = housing_df.drop(columns=cols_to_drop)


#### 4.2: Dropping features with high missingness

`EnclosedPorch` and `WoodDeckSF` have more than 89–90% missing values.  
With so little information available, these features are unlikely to contribute meaningfully to the model and could introduce noise.

Therefore, I dropped these columns from housing_df using `drop(columns=cols_to_drop)`.

In [29]:
# 4.3 Impute numeric features with the median

numeric_median_cols = ["LotFrontage", "BedroomAbvGr", "2ndFlrSF", "MasVnrArea", "GarageYrBlt"]

for col in numeric_median_cols:
    median_value = housing_df[col].median()
    housing_df[col] = housing_df[col].fillna(median_value)

#### 4.3 Imputing numeric features using the median (with justification from summary statistics)

For the numeric features with missing values (`LotFrontage`, `BedroomAbvGr`, `2ndFlrSF`, `MasVnrArea`, and `GarageYrBlt`), I used the **median** to impute missing values.

This decision is based on the summary statistics generated earlier (`df.select_dtypes(include=['number']).describe().T`), which show that several of these columns are **skewed** or contain **extreme values**:

- **LotFrontage**  
  - Range: from 21.0 to 313.0 
  - Mean: 70.0, Median: 69.0
  - The range indicates the presence of unusually large frontage values, suggesting outliers.

- **2ndFlrSF**  
  - Many houses have 0 second-floor square footage (no second floor).  
  - Maximum reaches 2065, showing high skewness.

- **MasVnrArea**  
  - Median = 0.0 but max = 1600.0 
  - This confirms the column is highly skewed, with most houses having no masonry veneer but a few having very large values.

- **GarageYrBlt**  
  - Some values are missing because some properties have no garage.  
  - The range spans more than 100 years, with outliers possible from very old or very new garages.

Using the **mean** (average) would shift these features upward because outliers inflate the mean.  
Using the **median** is more appropriate because:

- It is robust to outliers and skewed data.  
- It reflects the typical house in the dataset more accurately than the mean.  
- It preserves the distribution of these features without artificially inflating them.

Therefore, imputing missing values with the **median** ensures that:
- The model is not biased by extreme values.
- The imputation reflects the central tendency of each feature.
- The cleaned dataset remains representative of the underlying housing market.


In [33]:
# 4.4 Impute categorical features where missing means "no feature"

housing_df["GarageFinish"] = housing_df["GarageFinish"].fillna("No Garage")
housing_df["BsmtFinType1"] = housing_df["BsmtFinType1"].fillna("No Basement")
housing_df["BsmtExposure"] = housing_df["BsmtExposure"].fillna("No Basement")

#### 4.4 Imputing categorical features where missing values indicate “absence” of the feature

For the categorical features `GarageFinish`, `BsmtFinType1`, and `BsmtExposure`, I replaced missing values with descriptive labels e.g. "No Garage” for `GarageFinish`. This decision is based on both the meaning of the features and the patterns shown in the dataset.

##### Why these columns should not be imputed with the mode

From the descriptive summary and the nature of these variables, it is clear that missing values do not represent unknown information. Instead, they represent the absence of the feature itself:

- A house may genuinely not have a garage, meaning there is no value for `GarageFinish`.
- A house may not have a basement, meaning features such as `BsmtFinType1` and `BsmtExposure` cannot logically have a value.

If I used the mode (most frequent category), I would incorrectly assign a value such as:
- “Unf” or “RFn” to a house that does not have a garage, or
- “No” or “Gd” to a house with no basement.

This would misrepresent the property and introduce incorrect information into the dataset, which could reduce model accuracy.

##### Evidence from missing value patterns

Looking at the missing value table:

| Feature        | Missing % |
|----------------|-----------|
| GarageFinish   | 16.10%    |
| BsmtFinType1   | 9.93%     |
| BsmtExposure   | 2.60%     |

The missingness in these specific columns is aligned with the number of properties that also lack values in other garage/basement-related columns (e.g., `GarageYrBlt`, basement square footage columns). This suggests that missingness is not random—it is structural and directly tied to the physical characteristics of the house.

Using explicit labels improves clarity:

- `"No Garage"` clearly distinguishes houses with no garage from those with unfinished or finished garages.
- `"No Basement"` distinguishes houses with no basement from those where the basement exists but has certain finishing or exposure conditions.

These labels allow the machine learning model to correctly learn that the absence of a basement or garage is itself a feature that may influence the sale price.

##### Why this approach benefits the model

- It preserves all rows in the dataset (no data loss).
- It prevents incorrect assumptions about basement/garage quality.
- It provides the model with a high-quality, informative categorical feature showing whether the property has or doesn’t have these structures.
- It aligns with standard handling of structural missingness in housing datasets.

Therefore, imputing these missing values with descriptive labels is the most accurate and meaningful approach for the Ames Housing dataset.

In [40]:
# 4.5 Re-check missing values after cleaning

missing_counts_after = housing_df.isnull().sum()
missing_table_after = (
    pd.DataFrame({
        "missing_count": missing_counts_after,
        "missing_percent": (missing_counts_after / len(housing_df)) * 100
    })
    .query("missing_count > 0")
    .sort_values(by="missing_percent", ascending=False)
)

# Check if there are any missing values
if missing_table_after.empty:
    print("No missing values found!")
else:
    display(missing_table_after)

No missing values found!


#### 4.5 Re-check missing values after cleaning

After applying all cleaning steps, I re-ran the missing values summary.

The condition `missing_table_after.empty` evaluated to `True`, so the message **"No missing values found!"** was printed. This confirms that:

- All missing values in the main `housing_df` dataset have been successfully handled.
- The dataset is now ready for further preparation (e.g. encoding, scaling) and model training.


### Step 5: Validate and correct data types

After handling missing values, the next step is to validate the data types of each feature.  
Correct data types are important because:

- Numerical features must be stored as numbers so they can be used in summary statistics and ML models.
- Categorical features should be stored as `object` or `category`, not as arbitrary numbers.
- Year-based columns should contain realistic values e.g. no dates in the far future.

In this step I will:
- Check the current data types in `housing_df`.
- Detect any features that might be stored as `object` but actually contain numeric values.
- Check year-based columns (e.g. `YearBuilt`, `GarageYrBlt`, `YearRemodAdd`) to ensure there are no impossible dates.

In [48]:
# 5.2 Display all column data types
housing_df.dtypes


1stFlrSF          int64
2ndFlrSF        float64
BedroomAbvGr    float64
BsmtExposure     object
BsmtFinSF1        int64
BsmtFinType1     object
BsmtUnfSF         int64
GarageArea        int64
GarageFinish     object
GarageYrBlt     float64
GrLivArea         int64
KitchenQual      object
LotArea           int64
LotFrontage     float64
MasVnrArea      float64
OpenPorchSF       int64
OverallCond       int64
OverallQual       int64
TotalBsmtSF       int64
YearBuilt         int64
YearRemodAdd      int64
SalePrice         int64
dtype: object

In [49]:
housing_df.dtypes.value_counts()

int64      13
float64     5
object      4
Name: count, dtype: int64

#### 5.2 Inspect current data types

The output above shows the distribution of data types in the cleaned dataset and lists the dtypes

| dtype        | Meaning                                                      |
| ------------ | ------------------------------------------------------------ |
| `int64`      | whole numbers (e.g., bedroom counts, year built)             |
| `float64`    | decimal numbers (e.g., lot frontage, sale price)             |
| `object`     | text/categorical data (e.g., garage finish, kitchen quality) |


Here I check that:
- Numerical variables (areas, counts, prices, etc.) are stored as integer or float types.
- Categorical variables (e.g. neighbourhood, exterior material, garage finish) are stored as `object`.
If any numeric-looking variables appear as `object`, they will be investigated in the next step.

In the main dataset, all numerical features appear correctly as `int64` or `float64`, and all categorical features appear as `object`. Therefore, no dtype corrections were required.


In [52]:
# 5.3 Inspect year-based columns

year_cols = [col for col in housing_df.columns if "Year" in col or "Yr" in col]
housing_df[year_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
GarageYrBlt,1460.0,1978.589041,23.997022,1900.0,1962.0,1980.0,2001.0,2010.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.0,1973.0,2000.0,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.0,1994.0,2004.0,2010.0


#### 5.3 Inspect year-based columns

The summary statistics for all year-based columns (`YearBuilt`, `GarageYrBlt`, and `YearRemodAdd`) show that the values fall within realistic historical ranges for residential properties in Ames, Iowa.

- The earliest construction year is 1872, which is reasonable for older homes.
- Garage construction years range from 1900 to 2010.
- Remodel years range from 1950 to 2010.

There are no impossible values (such as year 0, years far in the future, or non-numeric entries). All year-based columns use the correct numeric dtype (`int64` or `float64` after imputation), and therefore no additional corrections are required.

### Step 6: Outlier Detection and Handling

Outliers are extreme values that differ significantly from the rest of the dataset. 

Outliers can negatively impact regression models by:
- skewing the distribution of numeric features,
- affecting the learned relationship between features and the target,
- reducing model performance and stability.

In this step, I will:
- Identify numerical features with potential outliers,
- Use the Interquartile Range (IQR) method to quantify outliers,
- Decide whether to keep, cap, or remove outlier values depending on the feature's meaning in the housing context.

In [53]:
# Detect outliers for all numeric columns using IQR

numeric_cols = housing_df.select_dtypes(include=['int64', 'float64']).columns

outlier_summary = {}

for col in numeric_cols:
    Q1 = housing_df[col].quantile(0.25)
    Q3 = housing_df[col].quantile(0.75)
    IQR = Q3 - Q1

    # IQR rule for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = housing_df[(housing_df[col] < lower_bound) | (housing_df[col] > upper_bound)]
    outlier_summary[col] = len(outliers)

outlier_df = pd.DataFrame.from_dict(outlier_summary, orient='index', columns=['outlier_count'])
outlier_df.sort_values(by='outlier_count', ascending=False)

,outlier_count
OverallCond,125
LotFrontage,106
MasVnrArea,98
OpenPorchSF,77
LotArea,69
TotalBsmtSF,61
SalePrice,61
BedroomAbvGr,34
GrLivArea,31
BsmtUnfSF,29


### Interpreting Outlier Results

The table above summarises the number of statistical outliers present in each numerical feature, based on the Interquartile Range (IQR) rule:

- Lower bound = Q1 − 1.5 × IQR  
- Upper bound = Q3 + 1.5 × IQR  

Any value outside this range is flagged as an outlier.

From the results, several features contain a meaningful number of outliers, such as:

- **OverallCond (125 outliers)**  
- **LotFrontage (106 outliers)**  
- **MasVnrArea (98 outliers)**  
- **LotArea (69 outliers)**  
- **SalePrice (61 outliers)**  
- **GrLivArea (31 outliers)**  

In the context of housing data, these outliers are not errors but instead reflect legitimate variations in property size, condition, or value. Real estate data naturally contains extreme values (e.g., unusually large homes, large parcels of land, or very high-value properties).

Removing these observations would incorrectly exclude rare but valid properties from the dataset. Since the goal of this project is to train a model that generalises well to all types of houses, decided not to remove or cap any outliers. This preserves the full variability of the housing market and allows the model to learn from both typical and extreme cases.

I Decided to retain all outliers as they represent legitimate property variations

### Step 7: Standardise and clean categorical values

Although so far our dataset is generally well structured, it is important to ensure that categorical features contain consistent and meaningful values. In this step, I check for:

- inconsistent spellings or formatting,
- unexpected values that may have appeared during data entry,
- placeholder values such as "NA" or "Missing",
- consistency between the main house_prices_records.csv dataset and the inherited_houses.csv dataset.

Since missing values were already handled earlier, this step focuses on verifying that all categories are clean and consistent.


In [55]:
# 7.2 Check unique categorical values to ensure consistency

categorical_cols = ["GarageFinish", "BsmtExposure", "BsmtFinType1", "KitchenQual"]

for col in categorical_cols:
    print(f"Column: {col}")
    print(housing_df[col].unique())
    print("-" * 40)

Column: GarageFinish
['RFn' 'Unf' 'No Garage' 'Fin']
----------------------------------------
Column: BsmtExposure
['No' 'Gd' 'Mn' 'Av' 'No Basement']
----------------------------------------
Column: BsmtFinType1
['GLQ' 'ALQ' 'Unf' 'Rec' 'No Basement' 'BLQ' 'LwQ']
----------------------------------------
Column: KitchenQual
['Gd' 'TA' 'Ex' 'Fa']
----------------------------------------


### 7.2 Inspecting categorical values for consistency

To ensure that all categorical features were clean and consistently formatted, I inspected the unique values for each categorical column:

- `GarageFinish`
- `BsmtExposure`
- `BsmtFinType1`
- `KitchenQual`

The outputs showed that all categories match the official values used in the Ames Housing dataset:

- **GarageFinish** contained only valid labels ("Unf", "RFn", "Fin") and the imputed value "No Garage".
- **BsmtExposure** and **BsmtFinType1** included only the recognised basement-related codes, plus the imputed category "No Basement".
- **KitchenQual** showed the expected quality ratings ("Ex", "Gd", "TA", "Fa").

There were no unexpected spellings, symbols, mixed types, or placeholder strings (e.g., "NA", "Missing").  

Therefore, no additional standardisation was necessary.

In [56]:
# 7.3 Check inherited_houses.csv consistency

for col in categorical_cols:
    if col in inherited_df.columns:
        print(f"Column: {col} (Inherited Houses)")
        print(inherited_df[col].unique())
        print("-" * 40)

Column: GarageFinish (Inherited Houses)
['Unf' 'Fin']
----------------------------------------
Column: BsmtExposure (Inherited Houses)
['No']
----------------------------------------
Column: BsmtFinType1 (Inherited Houses)
['Rec' 'ALQ' 'GLQ']
----------------------------------------
Column: KitchenQual (Inherited Houses)
['TA' 'Gd']
----------------------------------------


### 7.3 Checking categorical consistency in inherited_houses.csv

To ensure that the client’s inherited_houses.csv dataset was fully compatible with the main Ames Housing dataset, I inspected the unique categorical values for all relevant features (`GarageFinish`, `BsmtExposure`, `BsmtFinType1`, `KitchenQual`).

The output showed that the inherited houses only contain valid Ames Housing categories:

- The values are a proper subset of the categories found in the main dataset.  
- There were no inconsistencies, spelling variations, unexpected category labels, or missing/placeholder values.

Because both datasets share the same categorical structure, no additional transformation or standardisation is required. This ensures that the inherited houses can be used directly in downstream analysis and prediction without causing errors or misalignment.

### Step 8: Produce and save the clean dataset

After completing all cleaning steps, the final task in this notebook is to export the cleaned dataset.

Saving the cleaned data separately from the raw data helps to:

- keep a clear distinction between raw and processed datasets,
- make the workflow reproducible,
- allow the next notebooks (EDA and Model Training) to load a ready-to-use dataset.

The cleaned main Ames Housing dataset will be saved as `cleaned_data.csv` in the `data/processed/` folder.


---

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit

print("✅ Notebook connected to correct environment!")


ModuleNotFoundError: No module named 'sklearn'